In [1]:
!pip install gluonts
!pip install pytorchts

  Using cached holidays-0.11.1-py3-none-any.whl (133 kB)
  Using cached toolz-0.11.1-py3-none-any.whl (55 kB)
  Using cached convertdate-2.3.2-py3-none-any.whl (47 kB)


You should consider upgrading via the 'd:\github\gefcom14-s-comparison\venv\scripts\python.exe -m pip install --upgrade pip' command.


  Using cached hijri_converter-2.1.1-py3-none-any.whl (14 kB)
  Using cached korean_lunar_calendar-0.2.1-py3-none-any.whl (8.0 kB)
  Using cached PyMeeus-0.5.11.tar.gz (5.4 MB)
Using legacy 'setup.py install' for pymeeus, since package 'wheel' is not installed.
    Running setup.py install for pymeeus: started
    Running setup.py install for pymeeus: finished with status 'done'
  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached pyasn1_modules-0.2.8-py2.py3-none-any.whl (155 kB)
  Using cached requests_oauthlib-1.3.0-py2.py3-none-any.whl (23 kB)
  Using cached pyasn1-0.4.8-py2.py3-none-any.whl (77 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached certifi-2020.12.5-py2.py3-none-any.whl (147 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached oauthlib-3.1.0-py2.py3-none-any.whl (147 kB)

You should consider upgrading via the 'd:\github\gefcom14-s-comparison\venv\scripts\python.exe -m pip install --upgrade pip' command.



Using legacy 'setup.py install' for pytorchts, since package 'wheel' is not installed.
Using legacy 'setup.py install' for promise, since package 'wheel' is not installed.
Using legacy 'setup.py install' for subprocess32, since package 'wheel' is not installed.
Using legacy 'setup.py install' for pathtools, since package 'wheel' is not installed.
    Running setup.py install for subprocess32: started
    Running setup.py install for subprocess32: finished with status 'done'
    Running setup.py install for promise: started
    Running setup.py install for promise: finished with status 'done'
    Running setup.py install for pathtools: started
    Running setup.py install for pathtools: finished with status 'done'
    Running setup.py install for pytorchts: started
    Running setup.py install for pytorchts: finished with status 'done'


In [2]:
import numpy as np
import pandas as pd
import gluonts
import torch

In [3]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from pts.model.tempflow import TempFlowEstimator
from pts.model.transformer_tempflow import TransformerTempFlowEstimator
from pts import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

/usr/local/lib/python3.7/dist-packages/gluonts/json.py:46: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  "Using `json`-module for json-handling. "


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

## Prepeare data set

In [5]:
dataset = get_dataset("solar_nips", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/solar_nips/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/solar_nips/test/data.json


In [6]:
dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='137')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [7]:
train_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                   max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

In [8]:
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

/usr/local/lib/python3.7/dist-packages/gluonts/dataset/multivariate_grouper.py:182: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return {FieldName.TARGET: np.array([funcs(data) for data in dataset])}


## Evaluator

In [9]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})

## `GRU-Real-NVP`

In [10]:
estimator = TempFlowEstimator(
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    cell_type='GRU',
    input_size=552,
    freq=dataset.metadata.freq,
    scaling=True,
    dequantize=True,
    n_blocks=4,
    trainer=Trainer(device=device,
                    epochs=45,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100,
                    batch_size=64)
)

In [11]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:21,  4.66it/s, avg_epoch_loss=27.7, epoch=0]
99it [00:20,  4.74it/s, avg_epoch_loss=-111, epoch=1]
99it [00:20,  4.78it/s, avg_epoch_loss=-137, epoch=2]
99it [00:20,  4.77it/s, avg_epoch_loss=-143, epoch=3]
99it [00:20,  4.82it/s, avg_epoch_loss=-143, epoch=4]
99it [00:20,  4.85it/s, avg_epoch_loss=-141, epoch=5]
99it [00:20,  4.86it/s, avg_epoch_loss=-137, epoch=6]
99it [00:20,  4.81it/s, avg_epoch_loss=-101, epoch=7]
99it [00:20,  4.87it/s, avg_epoch_loss=-129, epoch=8]
99it [00:20,  4.86it/s, avg_epoch_loss=-131, epoch=9]
99it [00:20,  4.78it/s, avg_epoch_loss=-148, epoch=10]
99it [00:20,  4.78it/s, avg_epoch_loss=-156, epoch=11]
99it [00:20,  4.76it/s, avg_epoch_loss=-163, epoch=12]
99it [00:20,  4.80it/s, avg_epoch_loss=-169, epoch=13]
99it [00:20,  4.83it/s, avg_epoch_loss=-171, epoch=14]
99it [00:20,  4.81it/s, avg_epoch_loss=-176, epoch=15]
99it [00:20,  4.81it/s, avg_epoch_loss=-175, epoch=16]
99it [00:20,  4.84it/s, avg_epoch_loss=-179, epoch=17]
99it [00:20,  4.74it

### Metrics

In [12]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.4354353082379938
ND: 0.5441695457837461
NRMSE: 1.0811582813390421
MSE: 1108.8079876990867


In [13]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 0.388968382038156
ND-Sum: 0.47992752266206956
NRMSE-Sum: 0.8563455028685072
MSE-Sum: 13056205.761904761


## `GRU-MAF`

In [14]:
estimator = TempFlowEstimator(
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    cell_type='GRU',
    input_size=552,
    freq=dataset.metadata.freq,
    scaling=True,
    dequantize=True,
    flow_type='MAF',
    trainer=Trainer(device=device,
                    epochs=25,
                    learning_rate=1e-3,
                    num_batches_per_epoch=100,
                    batch_size=64)
)

In [15]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:18,  5.38it/s, avg_epoch_loss=101, epoch=0]
99it [00:18,  5.44it/s, avg_epoch_loss=-116, epoch=1]
99it [00:18,  5.36it/s, avg_epoch_loss=-171, epoch=2]
99it [00:18,  5.41it/s, avg_epoch_loss=-189, epoch=3]
99it [00:18,  5.39it/s, avg_epoch_loss=-200, epoch=4]
99it [00:18,  5.40it/s, avg_epoch_loss=-204, epoch=5]
99it [00:18,  5.41it/s, avg_epoch_loss=-208, epoch=6]
99it [00:18,  5.40it/s, avg_epoch_loss=-212, epoch=7]
99it [00:18,  5.42it/s, avg_epoch_loss=-217, epoch=8]
99it [00:18,  5.44it/s, avg_epoch_loss=-217, epoch=9]
99it [00:18,  5.41it/s, avg_epoch_loss=-220, epoch=10]
99it [00:18,  5.39it/s, avg_epoch_loss=-221, epoch=11]
99it [00:18,  5.43it/s, avg_epoch_loss=-223, epoch=12]
99it [00:18,  5.47it/s, avg_epoch_loss=-224, epoch=13]
99it [00:18,  5.37it/s, avg_epoch_loss=-225, epoch=14]
99it [00:18,  5.34it/s, avg_epoch_loss=-227, epoch=15]
99it [00:18,  5.33it/s, avg_epoch_loss=-227, epoch=16]
99it [00:18,  5.38it/s, avg_epoch_loss=-227, epoch=17]
99it [00:18,  5.39it/

### Metrics

In [16]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.4539193228769728
ND: 0.5683486104008995
NRMSE: 1.1302776657048887
MSE: 1211.8478019682525


In [17]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 0.4286909997228526
ND-Sum: 0.5315364158233865
NRMSE-Sum: 0.9158534199597169
MSE-Sum: 14933819.095238095


## `Transformer-MAF`

In [18]:
estimator = TransformerTempFlowEstimator(
    d_model=16,
    num_heads=4,
    input_size=552,
    target_dim=int(dataset.metadata.feat_static_cat[0].cardinality),
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length*4,
    flow_type='MAF',
    dequantize=True,
    freq=dataset.metadata.freq,
    trainer=Trainer(
        device=device,
        epochs=14,
        learning_rate=1e-3,
        num_batches_per_epoch=100,
        batch_size=64,
    )
)

In [19]:
predictor = estimator.train(dataset_train)
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)

agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

99it [00:22,  4.42it/s, avg_epoch_loss=45.1, epoch=0]
99it [00:22,  4.44it/s, avg_epoch_loss=-160, epoch=1]
99it [00:22,  4.38it/s, avg_epoch_loss=-191, epoch=2]
99it [00:22,  4.45it/s, avg_epoch_loss=-207, epoch=3]
99it [00:22,  4.42it/s, avg_epoch_loss=-217, epoch=4]
99it [00:22,  4.46it/s, avg_epoch_loss=-221, epoch=5]
99it [00:22,  4.44it/s, avg_epoch_loss=-226, epoch=6]
99it [00:22,  4.42it/s, avg_epoch_loss=-228, epoch=7]
99it [00:22,  4.45it/s, avg_epoch_loss=-231, epoch=8]
99it [00:22,  4.45it/s, avg_epoch_loss=-233, epoch=9]
99it [00:22,  4.42it/s, avg_epoch_loss=-234, epoch=10]
99it [00:22,  4.43it/s, avg_epoch_loss=-235, epoch=11]
99it [00:22,  4.44it/s, avg_epoch_loss=-236, epoch=12]
99it [00:22,  4.40it/s, avg_epoch_loss=-237, epoch=13]
Running evaluation: 7it [00:00, 51.17it/s]
Running evaluation: 7it [00:00, 48.80it/s]
Running evaluation: 7it [00:00, 44.03it/s]
Running evaluation: 7it [00:00, 46.71it/s]
Running evaluation: 7it [00:00, 48.42it/s]
Running evaluation: 7it [

### Metrics

In [20]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.3778790538056557
ND: 0.5088008270871934
NRMSE: 0.9849331182781657
MSE: 920.219154717236


In [21]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 0.32848166435879894
ND-Sum: 0.43418536499434257
NRMSE-Sum: 0.7728329325889608
MSE-Sum: 10633841.523809524
